In [ ]:
net = GCN(g=g, 
                m_fea_dim=m_fea_dim, 
                num_hidden=128, 
                num_labels=num_labels, 
                num_layers=args.num_layers,
                dropout=args.dropout,
                device=device)

# Test model
net.load_state_dict(torch.load('checkpoint.pt'))
net.eval()
with torch.no_grad():
    output_labels = labels[test_idx].cpu().numpy()
    output_predictions = net.inference(node_count_by_type, features_list)
    output_predictions = output_predictions.view(-1)[test_idx].cpu().numpy()
    km_r2 = r2_score(output_labels, output_predictions)
    print('R2 score for km: {}'.format(km_r2))